In [404]:
import pandas               as pd
import numpy                as np
import plotly.graph_objects as go
import plotly.express       as px
import plotly.io            as pio
from scipy import stats
from statsmodels.tsa.stattools import acf, adfuller
from statsmodels.tsa.arima_process import arma_generate_sample
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.distributions.empirical_distribution import ECDF
from distfit import distfit
from scipy.special import inv_boxcox
from scipy.optimize          import curve_fit

In [405]:
np.random.seed(12345)

In [406]:
df = pd.read_excel('Dziki_data_total.xlsx')
df.head()

,Data potwierdzenia,Liczba dzików dodatnich
0,2020-02-21 00:00:00,1
1,2021-01-04 00:00:00,32
2,2021-01-05 00:00:00,23
3,2021-01-07 00:00:00,1
4,2021-01-08 00:00:00,13


In [407]:
df['Data potwierdzenia'] = pd.to_datetime(df['Data potwierdzenia'])
df['Data potwierdzenia'] = df['Data potwierdzenia'].apply(lambda x: x.date())
df = df.sort_values(by='Data potwierdzenia')
df = df.drop(0)
df = df.reset_index(drop=True)
df.head()

,Data potwierdzenia,Liczba dzików dodatnich
0,2021-01-04,32
1,2021-01-05,23
2,2021-01-07,1
3,2021-01-08,13
4,2021-01-09,44


In [408]:
df['Data potwierdzenia'] = pd.to_datetime(df['Data potwierdzenia'])
df.set_index('Data potwierdzenia', inplace=True)
df

,Liczba dzików dodatnich
Data potwierdzenia,
2021-01-04,32
2021-01-05,23
2021-01-07,1
2021-01-08,13
2021-01-09,44
...,...
2024-03-14,26
2024-03-15,21
2024-03-18,11


In [409]:
# df = df.resample('W-Mon').sum()
df = df.resample('W').sum()

In [410]:
fig = px.line(
    df,
    x=df.index, 
    y="Liczba dzików dodatnich"
)


fig.update_layout(
    title={
        'text': f"Liczba dzików dodatnich w czasie",
        'x': 0.5,  
        'xanchor': 'center'
    },
    xaxis_title="Data",
    yaxis_title="Liczba dzików dodatnich",
    title_font_size=18,
    width=1000,
    height=500,
    showlegend=False
)

fig.show()

In [411]:
len(df.index)

168

In [412]:
end_date = pd.to_datetime('2023-12-31')
filtered_df = df[df.index <= end_date]


cut_df = pd.DataFrame({
    "Liczba dzików dodatnich": filtered_df['Liczba dzików dodatnich']
})

cut_df.tail()

,Liczba dzików dodatnich
Data potwierdzenia,
2023-12-03,56
2023-12-10,61
2023-12-17,49
2023-12-24,62
2023-12-31,44


In [413]:
rest_df =  df[df.index >= end_date]

predict_df = pd.DataFrame({
    "Liczba dzików dodatnich": rest_df['Liczba dzików dodatnich']
})

predict_df

,Liczba dzików dodatnich
Data potwierdzenia,
2023-12-31,44
2024-01-07,18
2024-01-14,73
2024-01-21,65
2024-01-28,43
2024-02-04,41
2024-02-11,77
2024-02-18,33
2024-02-25,75


In [414]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=cut_df.index, 
        y=cut_df["Liczba dzików dodatnich"],  
        mode='lines',
        name='Dane do analizy'
    )
)

fig.add_trace(
    go.Scatter(
        x=predict_df.index,
        y=predict_df["Liczba dzików dodatnich"],
        name="Dane do predykcji",
        mode="lines",
        line=dict(color='red')
    )
)


fig.update_layout(
    title={
        'text': "Wydzielone dane na predykcyjne i do analizy",
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title="Data",
    yaxis_title="Liczba dzików dodatnich",
    title_font_size=18,
    width=1000,
    height=500,
    legend=dict(
        orientation='h',
        yanchor='bottom',
        xanchor='center',
        x=0.5,
        y=-0.3
    ),
)

fig.show()

In [415]:
bc_boars, lam = stats.boxcox(cut_df["Liczba dzików dodatnich"])
cut_df["dane boxcox"] = bc_boars

# cut_df["zróżnicowane dane"] = cut_df["dane boxcox"].diff()

In [416]:
fig = px.line(
    cut_df, 
    x=cut_df.index, 
    y="dane boxcox"
)

fig.update_layout(
    title={
        'text': f"Dane po transformacji boxacoxa z lambda = {lam}",
        'x': 0.5,  
        'xanchor': 'center'
    },
    xaxis_title="Data",
    yaxis_title="Otrzymane wartości",
    title_font_size=18,
    width=1000,
    height=500,
    showlegend=False
)

fig.show()

In [417]:
def func_sin(x, a, b, c):
    return a*np.sin(b*x ) + c


max_value = np.max(cut_df["dane boxcox"])
period_estimate = 55 # Przykładowa wartość okresu oscylacji
mean_value = np.mean(cut_df["dane boxcox"])
p0 = [max_value, 2*np.pi/period_estimate, mean_value] 
print(p0)
# p0 = [10, 0.15, 5]

# max_value = np.max(cut_df["dane boxcox"])
# period_estimate = 55  # Przykładowa wartość okresu oscylacji
# mean_value = np.mean(cut_df["dane boxcox"])
# phase_estimate = np.arcsin((cut_df["dane boxcox"] - mean_value) / max_value)
# p0 = [max_value, 2 * np.pi / period_estimate, np.mean(phase_estimate), mean_value]
# print(p0)


popt_sin, pcov_sin = curve_fit(func_sin, range(len(cut_df.index)), cut_df["dane boxcox"], p0=p0)
popt_sin

[7.923414966170309, 0.11423973285781065, 5.229949887564612]


array([1.08281935, 0.1268186 , 5.21057639])

In [418]:
popt_sin, pcov_sin = curve_fit(func_sin, range(len(cut_df.index)), cut_df["dane boxcox"], p0=p0)
popt_sin

array([1.08281935, 0.1268186 , 5.21057639])

In [419]:
sin_data = func_sin(range(len(cut_df.index)), *popt_sin)

In [420]:
fig = px.line(
    cut_df, 
    x=cut_df.index, 
    y="dane boxcox"
)

fig.add_scatter(
    x=cut_df.index,
    y=sin_data, 
    mode='lines', 
    line=dict(color='red', width=2)
)


fig.update_layout(
    title={
        'text': "Dobranie sinusa do danych",
        'x': 0.5,  
        'xanchor': 'center'
    },
    xaxis_title="Data",
    yaxis_title="Otrzymane wartości",
    title_font_size=18,
    width=1000,
    height=500,
    showlegend=False
)

fig.show()

In [421]:
cut_df["zróżnicowane dane"] = cut_df["dane boxcox"] - sin_data

In [422]:
fig = px.line(
    cut_df, 
    x=cut_df.index, 
    y="zróżnicowane dane"
)


fig.update_layout(
    title={
        'text': "Dane w postaci stacjonarnej",
        'x': 0.5,  
        'xanchor': 'center'
    },
    xaxis_title="Data",
    yaxis_title="Otrzymane wartości",
    title_font_size=18,
    width=1000,
    height=500,
    showlegend=False
)

fig.show()

In [423]:
print(f'Ile mamy NaN: {sum(cut_df["zróżnicowane dane"].isna())}')
print(f'Indeks gdzie mamy NaN: {cut_df.loc[cut_df["zróżnicowane dane"].isna()].index}')
cut_df.head()

Ile mamy NaN: 0
Indeks gdzie mamy NaN: DatetimeIndex([], dtype='datetime64[ns]', name='Data potwierdzenia', freq='W-SUN')


,Liczba dzików dodatnich,dane boxcox,zróżnicowane dane
Data potwierdzenia,,,
2021-01-10,113,6.414482,1.203906
2021-01-17,139,6.789929,1.442399
2021-01-24,213,7.594624,2.112340
2021-01-31,118,6.492189,0.879515
2021-02-07,210,7.567196,1.830590


In [424]:
new_df = cut_df.dropna()
new_df.head()

,Liczba dzików dodatnich,dane boxcox,zróżnicowane dane
Data potwierdzenia,,,
2021-01-10,113,6.414482,1.203906
2021-01-17,139,6.789929,1.442399
2021-01-24,213,7.594624,2.112340
2021-01-31,118,6.492189,0.879515
2021-02-07,210,7.567196,1.830590


In [425]:
def adfuller_test(series, sig=0.05): # kod pliku prowadzącego
    res = adfuller(series, autolag='AIC')    
    p_value = round(res[1], 3) 
    stats   = round(res[0], 3) 

    if p_value <= sig:
        print(f"Statystyka testowa = {stats}, p-Value = {p_value} => Stationary. ")
    else:
        print(f"Statystyka testowa = {stats}, p-value = {p_value} => Non-stationary.")

In [426]:
adfuller_test(new_df["zróżnicowane dane"])

Statystyka testowa = -3.13, p-Value = 0.024 => Stationary. 


In [427]:
h = 100
acf_boars = acf(new_df["zróżnicowane dane"], nlags=h)

In [428]:
fig = go.Figure()

[fig.add_scatter(
    x=(x,x), 
    y=(0,acf_boars[x]), 
    mode='lines', 
    line_color='#767FFA'
    ) for x in range(len(acf_boars))
]

fig.add_trace(
    go.Scatter(
        x=np.arange(0, h + 1),
        y=acf_boars,
        mode='markers',
        line=dict(width=2, color = '#767FFA'),
        marker=dict(
            symbol='circle',
            size=8,
        )
    )
)

fig.update_layout(
    title={
        'text': "Funkcja autokorelacji dla danych",
        'x': 0.5,  
        'xanchor': 'center'
    },
    xaxis_title="h",
    yaxis_title="ACF",
    title_font_size=18,
    width=800,
    height=500,
    showlegend=False
)

fig.update_traces(showlegend=False)
fig.show()


In [429]:
from scipy.stats import spearmanr

def acf_spearman(data, lag):
    n = len(data)
    acf_values = []

    for i in range(lag + 1):
        x = data[:n - i]
        y = data[i:]
        rho, _ = spearmanr(x, y)
        acf_values.append(rho)

    return acf_values

resist_acf = np.array(2)*np.sin(acf_spearman(new_df["zróżnicowane dane"], h)*np.array(np.pi/6))

In [430]:
fig = go.Figure()

[fig.add_scatter(
    x=(x,x), 
    y=(0,resist_acf[x]), 
    mode='lines',
    line_color='#767FFA'
    ) for x in range(len(resist_acf))
]

fig.add_trace(
    go.Scatter(
        x=np.arange(0, h + 1),
        y=resist_acf,
        mode='markers',
        line=dict(width=2, color='#767FFA'),
        marker=dict(
            symbol='circle',
            size=8
        )
    )
)

fig.update_layout(
    title={
        'text': "Funkcja autokorelacji dla danych - estymator odporny",
        'x': 0.5,  
        'xanchor': 'center'
    },
    xaxis_title="h",
    yaxis_title="ACF",
    title_font_size=18,
    width=800,
    height=500,
    showlegend=False
)

fig.show()

In [431]:
def func(data, max_p, max_q):
    ps = np.zeros((max_p * max_q,))
    qs = np.zeros((max_p * max_q,))
    AIC = np.zeros((max_p * max_q,))
    BIC = np.zeros((max_p * max_q,))
    HQIC = np.zeros((max_p * max_q,))

    i = 0
    for p in range(0, max_p):
        for q in range(0, max_q):
            model = ARIMA(data, order=(p, 0, q))
            model_fit = model.fit()
            ps[i] = p
            qs[i] = q
            AIC[i] = model_fit.aic
            BIC[i] = model_fit.bic
            HQIC[i] = model_fit.hqic
            
            i += 1

    df = pd.DataFrame({'p': ps, 'q': qs, 'AIC': AIC, 'BIC': BIC, 'HQIC': HQIC})
    return df

In [432]:
df_params = func(new_df["zróżnicowane dane"], 16, 16)

In [433]:
df_params.sort_values(by='AIC').head()

,p,q,AIC,BIC,HQIC
51,3.0,3.0,406.049533,430.448382,415.959289
35,2.0,3.0,406.813966,428.162958,415.485002
118,7.0,6.0,407.249777,452.997617,425.830569
67,4.0,3.0,407.702344,435.151048,418.850819
52,3.0,4.0,408.191233,435.639937,419.339708


In [434]:
top_row_aic = df_params.sort_values(by='AIC').iloc[0]
p_aic = top_row_aic['p']
q_aic = top_row_aic['q']
print(p_aic, q_aic)

3.0 3.0


In [435]:
df_params.sort_values(by='BIC').head()

,p,q,AIC,BIC,HQIC
17,1.0,1.0,409.133872,421.333297,414.088750
18,1.0,2.0,411.133839,426.383119,417.327436
33,2.0,1.0,411.133847,426.383127,417.327445
35,2.0,3.0,406.813966,428.162958,415.485002
48,3.0,0.0,413.316813,428.566093,419.510410


In [436]:
top_row_bic = df_params.sort_values(by='BIC').iloc[0]
p_bic = top_row_bic['p']
q_bic = top_row_bic['q']
print(p_bic, q_bic)

1.0 1.0


In [437]:
df_params.sort_values(by='HQIC').head()

,p,q,AIC,BIC,HQIC
17,1.0,1.0,409.133872,421.333297,414.088750
35,2.0,3.0,406.813966,428.162958,415.485002
51,3.0,3.0,406.049533,430.448382,415.959289
18,1.0,2.0,411.133839,426.383119,417.327436
33,2.0,1.0,411.133847,426.383127,417.327445


In [438]:
top_row_hqic = df_params.sort_values(by='HQIC').iloc[0]
p_hqic = top_row_hqic['p']
q_hqic = top_row_hqic['q']
print(p_hqic, q_hqic)

1.0 1.0


In [439]:
model_aic = ARIMA(new_df["zróżnicowane dane"], order=(p_aic, 0, q_aic)).fit()
model_bic = ARIMA(new_df["zróżnicowane dane"], order=(p_bic, 0, q_bic)).fit() 
model_hqic = ARIMA(new_df["zróżnicowane dane"], order=(p_hqic, 0, q_hqic)).fit()

In [440]:
print(f'MSE dla AIC: {model_aic.mse}\n')
print(f'MSE dla BIC: {model_bic.mse}\n')
print(f'MSE dla HQIC: {model_hqic.mse}')
#  wybieram najmniejsze mse czyli model aic

MSE dla AIC: 0.716598062883144

MSE dla BIC: 0.7670447110720983

MSE dla HQIC: 0.7670447110720983


In [441]:
model_aic.params

const     0.056574
ar.L1     0.464492
ar.L2    -0.538948
ar.L3     0.932666
ma.L1    -0.179387
ma.L2     0.635637
ma.L3    -0.621315
sigma2    0.703654
dtype: float64

In [442]:
ar_coff_aic = model_aic.arparams 
ar_coff_aic = np.insert(-ar_coff_aic, 0, 1)

ma_coff_aic = model_aic.maparams
ma_coff_aic = np.insert(ma_coff_aic, 0, 1)

scale_aic = model_aic.params[-1]

print(ar_coff_aic)
print(ma_coff_aic)

[ 1.         -0.46449181  0.53894757 -0.93266565]
[ 1.         -0.17938666  0.63563683 -0.62131464]


In [443]:
arma_sample_aic = arma_generate_sample(ar_coff_aic, ma_coff_aic, nsample=len(new_df["zróżnicowane dane"]), scale = np.sqrt(scale_aic))
# arma_sample = model_aic.simulate(nsimulations=len(new_df["zróżnicowane dane"]))

In [444]:
fig = px.line(
    x=new_df.index, 
    y=arma_sample_aic
)


fig.update_layout(
    title={
        'text': f"Dopasowany model ARMA({p_aic}, {q_aic}) z WN({0}, {scale_aic})",
        'x': 0.5,  
        'xanchor': 'center'
    },
    xaxis_title="Data",
    yaxis_title="Otrzymane wartości",
    title_font_size=18,
    width=1000,
    height=500,
    showlegend=False
)

fig.show() 

In [445]:
def func_cf_interval(ar_coff, ma_coff, sigma, n, h, M, alph):
    est_acf = np.zeros((M, h + 1))
    
    for i in range(M):
        X_t = arma_generate_sample(ar_coff, ma_coff, nsample=n, scale = np.sqrt(sigma))
        est_acf[i] = acf(X_t, nlags = h)

    acf_up = np.zeros(h + 1)
    acf_down = np.zeros(h + 1)

    for k in range(h + 1):
        acf_up[k] = np.quantile(est_acf[:,k], 1 - (alph/2))
        acf_down[k] = np.quantile(est_acf[:,k], (alph/2))

    return acf_up, acf_down

In [446]:
up, down = func_cf_interval(ar_coff_aic, ma_coff_aic, scale_aic, len(new_df["zróżnicowane dane"]), h, 10000, 0.05)

In [494]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=np.arange(0, h + 1).tolist() + np.arange(0, h + 1).tolist()[::-1], 
        y=up.tolist() + down.tolist()[::-1], 
        fill='toself',
        fillcolor='rgba(155, 220, 108, 0.3)',
        line=dict(color='rgba(155, 220, 108, 0.45)'),
        name='Confidence Interval'
    )
)


[fig.add_scatter(
    x=(x,x), 
    y=(0,acf_boars[x]), 
    mode='lines', 
    line_color='#636efa'
    ) for x in range(len(acf_boars))
]

fig.add_trace(
    go.Scatter(
        x=np.arange(0, h + 1),
        y=acf_boars,
        mode='markers',
        line=dict(width=2, color = '#636efa'),
        marker=dict(
            symbol='circle',
            size=8,
        )
    )
)



fig.update_layout(
    title={
        'text': f"Funkcja autokorelacji dla danych z przedziałem ufności",
        'x': 0.5,  
        'xanchor': 'center'
    },
    xaxis_title="h",
    yaxis_title="ACF",
    title_font_size=18,
    width=800,
    height=500,
    showlegend=False
)

fig.show()


In [448]:
alpha = 0.05
X_t = np.zeros((1000, len(new_df["zróżnicowane dane"])))

for i in range(1000):
    X_t[i] = arma_generate_sample(ar_coff_aic, ma_coff_aic, nsample=len(new_df["zróżnicowane dane"]), scale = np.sqrt(scale_aic))

q1 = np.zeros(len(new_df["zróżnicowane dane"]))
q2 = np.zeros(len(new_df["zróżnicowane dane"]))

for k in range(len(new_df["zróżnicowane dane"])):
    q1[k] = np.quantile(X_t[:, k], (alpha/2))
    q2[k] = np.quantile(X_t[:, k], 1 - (alpha/2))

In [449]:
fig = px.line(
    x=new_df.index, 
    y=new_df["zróżnicowane dane"])


fig.add_scatter(
    x=new_df.index,
    y=q1, 
    mode='lines', 
    line=dict(color='red', width=2)
)

fig.add_scatter(
    x=new_df.index,
    y=q2, 
    mode='lines', 
    line=dict(color='red', width=2)
)


fig.update_layout(
    title={
        'text': f"Dane z liniami kwantylowymi z alpha = {alpha}",
        'x': 0.5,  
        'xanchor': 'center'
    },
    xaxis_title="Data",
    yaxis_title="Otrzymane wartości",
    title_font_size=18,
    width=1000,
    height=500,
    showlegend=False
)

fig.show()


In [450]:
res = model_aic.resid

In [495]:
fig = px.scatter(x=range(len(res)), y=res)

x_values = list(range(len(res)))

fig.add_scatter(
    x=x_values,
    y=np.array([-1.7]*len(res)), 
    mode='lines', 
    line=dict(color='black', width=2)
)

fig.add_scatter(
    x=x_values,
    y=np.array([1.7]*len(res)), 
    mode='lines', 
    line=dict(color='black', width=2)
)

fig.add_scatter(
    x=x_values,
    y=np.array([0]*len(res)), 
    mode='lines', 
    line=dict(color='red', width=2),
    name='E(epsilon)'
)


fig.update_layout(
    title={
        'text': "Wizualizacja residuów",
        'x': 0.5,  
        'xanchor': 'center'
    },
    xaxis_title="index",
    yaxis_title="Otrzymane wartości",
    title_font_size=18,
    width=800,
    height=500,
    showlegend=False
)

fig.update_yaxes(range=[-3, 3])

fig.show()


In [452]:
acf_res = acf(res, nlags=h)

In [453]:
fig = go.Figure()

[fig.add_scatter(x=(x,x), y=(0,acf_res[x]), mode='lines', line_color='#767FFA') 
     for x in range(len(acf_boars))]

fig.add_trace(
    go.Scatter(
        x=np.arange(0, h + 1),
        y=acf_res,
        mode='markers',
        line=dict(width=2, color = '#767FFA'),
        marker=dict(
            symbol='circle',
            size=8,
        )
    )
)

fig.update_layout(
    title={
        'text': "Funkcja autokorelacji dla residuów",
        'x': 0.5,  
        'xanchor': 'center'
    },
    xaxis_title="h",
    yaxis_title="ACF",
    title_font_size=18,
    xaxis=dict(
        tickfont_size=14
    ),
    yaxis=dict(
        tickfont_size=14
    ),
    width=800,
    height=500
)

fig.update_traces(showlegend=False)
fig.show()


In [454]:
# Test Kołmogorowa-Smirnowa
ks_test = stats.kstest((res - np.mean(res))/np.sqrt(scale_aic), "norm")

# Test Shapiro-Wilka
shapiro_test = stats.shapiro(res)

# Test Jarque-Bera
jarque_bera_test = stats.jarque_bera(res)


print("Test Kołmogorowa-Smirnowa:   " + str(ks_test))
print("Test Shapiro-Wilka:   " + str(shapiro_test))
print("Test Jarque-Bera:   " + str(jarque_bera_test)) # nie jest normalny

Test Kołmogorowa-Smirnowa:   KstestResult(statistic=0.09796075162537637, pvalue=0.09361350093454412)
Test Shapiro-Wilka:   ShapiroResult(statistic=0.9458855390548706, pvalue=1.0388900591351558e-05)
Test Jarque-Bera:   Jarque_beraResult(statistic=57.455677006256565, pvalue=3.339550858072471e-13)


In [455]:
dfit = distfit()
dfit.fit_transform(res)
# dfit.plot() 

[distfit] >INFO> fit
[distfit] >INFO> transform
[distfit] >INFO> [norm      ] [0.00 sec] [RSS: 0.106814] [loc=-0.030 scale=0.846]
[distfit] >INFO> [expon     ] [0.0 sec] [RSS: 0.991987] [loc=-3.353 scale=3.323]
[distfit] >INFO> [pareto    ] [0.23 sec] [RSS: 1.06271] [loc=-418604815.455 scale=418604812.102]
[distfit] >INFO> [dweibull  ] [0.03 sec] [RSS: 0.0543321] [loc=-0.023 scale=0.639]
[distfit] >INFO> [t         ] [0.08 sec] [RSS: 0.0497282] [loc=0.043 scale=0.636]
[distfit] >INFO> [genextreme] [0.17 sec] [RSS: 0.115084] [loc=-0.268 scale=0.917]
[distfit] >INFO> [gamma     ] [0.17 sec] [RSS: 0.128553] [loc=-16.477 scale=0.047]
[distfit] >INFO> [lognorm   ] [0.31 sec] [RSS: 0.105185] [loc=-562.877 scale=562.851]
[distfit] >INFO> [beta      ] [0.07 sec] [RSS: 0.105957] [loc=-4.514 scale=6.666]
[distfit] >INFO> [uniform   ] [0.0 sec] [RSS: 0.649222] [loc=-3.353 scale=5.112]
[distfit] >INFO> [loggamma  ] [0.04 sec] [RSS: 0.0822525] [loc=-0.574 scale=1.080]
[distfit] >INFO> Compute confi

{'model': {'name': 't',
  'score': 0.04972821263609057,
  'loc': 0.04292957980340624,
  'scale': 0.6358722936950958,
  'arg': (4.357400653738461,),
  'params': (4.357400653738461, 0.04292957980340624, 0.6358722936950958),
  'model': <scipy.stats._distn_infrastructure.rv_frozen at 0x1d5e6821660>,
  'bootstrap_score': 0,
  'bootstrap_pass': None,
  'color': '#e41a1c',
  'CII_min_alpha': -1.2813287220277936,
  'CII_max_alpha': 1.367187881634606},
 'summary':           name     score               loc             scale  \
 0            t  0.049728           0.04293          0.635872   
 1     dweibull  0.054332          -0.02346          0.639213   
 2     loggamma  0.082252         -0.574324          1.079676   
 3      lognorm  0.105185       -562.877165        562.851001   
 4         beta  0.105957         -4.514176          6.666302   
 5         norm  0.106814         -0.030057          0.845987   
 6   genextreme  0.115084         -0.268365          0.916559   
 7        gamma  0.12

In [497]:
def t_test(X): 
    t_stat, p_val = stats.ttest_1samp(a=X, popmean = 0)
    print("t-statistic = " + str(t_stat))  
    print("p-value = " + str(p_val)) 
    if p_val < 0.05:
        print('Odrzucamy hipotezę zerową, że średnia residuów jest równa 0')
    else:
        print('Nie mamy podstaw by odrzucić hipotezę zerową, że średnia residuów jest równa 0')

t_test(res)

t-statistic = -0.4423241177022889
p-value = 0.6588712664981433
Nie mamy podstaw by odrzucić hipotezę zerową, że średnia residuów jest równa 0


In [498]:
np.mean(res)

-0.03005651765493418

In [456]:
ecdf = ECDF(res)
t = np.linspace(np.min(res), np.max(res), 1000)
params = stats.t.fit(res)
t_student = stats.t(*params)
th_F = t_student.cdf(t)
th_f = t_student.pdf(t)
print(params)

(4.357400653738461, 0.04292957980340624, 0.6358722936950958)


In [457]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=ecdf.x, 
        y=ecdf.y, 
        mode='lines', 
        name='Dystrybuanta empiryczna'
    )
)

fig.add_trace(
    go.Scatter(
        x=t, 
        y=th_F, 
        mode='lines', 
        name=f'Dystrybuanta rozkładu t-studenta({round(params[0], 2)}, {round(params[1], 2)}, {round(params[2], 2)})',
        line=dict(dash='dash')  
    )
)


fig.update_layout(
    title={
        'text': 'Porównanie dystrubuant',
        'x': 0.5,  
        'xanchor': 'center'
    },
    xaxis_title='x',
    yaxis_title='F(x)',
    showlegend=True,
    width=800,
    height=500,
    legend=dict(
        orientation='h', 
        yanchor='bottom',  
        xanchor='center',  
        x=0.5,  
        y=-0.3)
)


fig.show()

In [458]:
fig = go.Figure()

fig.add_trace(
        go.Histogram(
        x=res, 
        histnorm='probability density', 
        name='Gęstość empiryczna'
    )
)

fig.add_trace(
    go.Scatter(
        x=t, 
        y=th_f, 
        mode='lines', 
        name=f'Gęstość rozkładu t-studenta({round(params[0], 2)}, {round(params[1], 2)}, {round(params[2], 2)})'
    )
)

fig.update_layout(
    title={
        'text': 'Porównanie gęstości',
        'x': 0.5,  
        'xanchor': 'center'
    },
    xaxis_title='x',
    yaxis_title='f(x)',
    showlegend=True,
    width=800,
    height=500,
    legend=dict(
        orientation='h', 
        yanchor='bottom',  
        xanchor='center',  
        x=0.5,  
        y=-0.3  
    )
)



fig.show()


# **PREDYKCJA 2024**

In [461]:
x_p = np.arange(len(new_df.index), len(new_df.index) + len(predict_df.index))

In [462]:
forecast = model_aic.get_forecast(steps=52) # 2024 bedzie miec 54 tygodnie

forecast_values = forecast.predicted_mean
confidence_intervals = forecast.conf_int()

x_p_all = np.arange(len(new_df.index), len(new_df.index) + 52)

transformed_forecast = forecast_values + func_sin(x_p_all, *popt_sin) 
transformed_forecast = inv_boxcox(transformed_forecast, lam)

interval_down =  confidence_intervals["lower zróżnicowane dane"] + func_sin(x_p_all, *popt_sin)
interval_up = confidence_intervals["upper zróżnicowane dane"] + func_sin(x_p_all, *popt_sin)

interval_down = inv_boxcox(interval_down, lam)
interval_up = inv_boxcox(interval_up, lam)

In [463]:
start_date = '2024-01-01'
end_date = '2024-12-31'

date_range = pd.date_range(start=start_date, end=end_date, freq='W')

date_range

DatetimeIndex(['2024-01-07', '2024-01-14', '2024-01-21', '2024-01-28',
               '2024-02-04', '2024-02-11', '2024-02-18', '2024-02-25',
               '2024-03-03', '2024-03-10', '2024-03-17', '2024-03-24',
               '2024-03-31', '2024-04-07', '2024-04-14', '2024-04-21',
               '2024-04-28', '2024-05-05', '2024-05-12', '2024-05-19',
               '2024-05-26', '2024-06-02', '2024-06-09', '2024-06-16',
               '2024-06-23', '2024-06-30', '2024-07-07', '2024-07-14',
               '2024-07-21', '2024-07-28', '2024-08-04', '2024-08-11',
               '2024-08-18', '2024-08-25', '2024-09-01', '2024-09-08',
               '2024-09-15', '2024-09-22', '2024-09-29', '2024-10-06',
               '2024-10-13', '2024-10-20', '2024-10-27', '2024-11-03',
               '2024-11-10', '2024-11-17', '2024-11-24', '2024-12-01',
               '2024-12-08', '2024-12-15', '2024-12-22', '2024-12-29'],
              dtype='datetime64[ns]', freq='W-SUN')

In [505]:
fig = go.Figure() 


fig.add_scatter(
    x=new_df.index, 
    y=new_df["zróżnicowane dane"],  
    mode='lines',
    name='Przekształcone dane z analizy (lata 2021-2023)'
    )

fig.add_trace(
    go.Scatter(
        x=date_range.tolist() + date_range.tolist()[::-1], 
        y=confidence_intervals["upper zróżnicowane dane"].tolist() + confidence_intervals["lower zróżnicowane dane"].tolist()[::-1],  
        fill='toself',
        fillcolor='rgba(155, 220, 108, 0.3)',
        line=dict(color='rgba(155, 220, 108, 0.45)'),
        name='Przedział ufności predykcji dla roku 2024'
    )
)


fig.add_scatter(
    x=predict_df.index,
    y=stats.boxcox(predict_df["Liczba dzików dodatnich"], lam) - func_sin(x_p, *popt_sin),
    mode='lines', 
    line=dict(width=2, color='red'),
    name="Przekształcone dane predykcyjne do marca 2024"
    )


fig.add_scatter(
    x=date_range,
    y=forecast_values, 
    mode='lines', 
    line=dict(color='black', width=3),
    name="Przewidywana średnia przekształconych danych dla roku 2024"
    )


fig.update_layout(
    title={
        'text': f"Predykcja danych po transformacji",
        'x': 0.5,  
        'xanchor': 'center'
    },
    xaxis_title="Data",
    yaxis_title="Otrzymane wartości",
    title_font_size=18,
    width=1000,
    height=500,
    showlegend=True,
    legend=dict(
        orientation='h', 
        yanchor='bottom',  
        xanchor='center',  
        x=0.5,  
        y=-0.3)
    
)

fig.show()

In [504]:
fig = go.Figure()

fig.add_scatter(
    x=new_df.index, 
    y=new_df["Liczba dzików dodatnich"],  
    mode='lines',
    name='Dane do analizy (lata 2021-2023)'
    )


fig.add_trace(
    go.Scatter(
        x=date_range.tolist() + date_range.tolist()[::-1], 
        y=interval_up.tolist() + interval_down.tolist()[::-1], 
        fill='toself',
        fillcolor='rgba(155, 220, 108, 0.3)',
        line=dict(color='rgba(155, 220, 108, 0.45)'),
        name='Przedział ufności predykcji dla roku 2024'
    )
)


fig.add_scatter(
    x=predict_df.index,
    y=predict_df["Liczba dzików dodatnich"],
    mode='lines', 
    line=dict(width=2, color='red'),
    name="Dane predykcyjne do marca 2024"
    )


fig.add_scatter(
    x=date_range,
    y=transformed_forecast, 
    mode='lines', 
    line=dict(color='black', width=3),
    name="Przewidywana średnia z danych dla 2024"
    )


fig.update_layout(
    title={
        'text': f"Predykcja danych przed transformacją",
        'x': 0.5,  
        'xanchor': 'center'
    },
    xaxis_title="Data",
    yaxis_title="Otrzymane wartości",
    title_font_size=18,
    width=1000,
    height=500,
    showlegend=True,
    legend=dict(
        orientation='h', 
        yanchor='bottom',  
        xanchor='center',  
        x=0.5,  
        y=-0.3)
)

fig.show()